# Scipy
SciPy (Scientific Python) è una libreria per la computazione scientifica costruita su NumPy (quindi tutte le strutture dati di NumPy possono essere usate con metodi di SciPy). Contiene varie funzioni per ottimizzazione, statistica e signal processing.

È molto diffusa e utilizzata perché contiene parecchie funzioni per l'ottimizzazione e il data fitting estremamente facili e intuitive da utilizzare, come vedremo più avanti.

*Curiosità: è stata creata dallo stesso creatore di NumPy.*

Come prima cosa importiamo SciPy e altre librerie che potrebbero tornare utili:

In [ ]:
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt

sp.__version__

Per quanto riguarda la documentazione, il discorso è analogo alle altre librerie:

In [ ]:
sp.optimize?

Nelle prossime sezioni vedremo i moduli e le funzioni principali di SciPy.

## Scipy Constants

Questo modulo contiene molto semplicemente le costanti scientifiche più utilizzate e varie unità di misura.

Per visualizzare un elenco di tutte le costanti presenti nel modulo:

In [ ]:
sp.constants.find()

Si utilizzano utilizzando le chiavi sopra visualizzate con il dizionario ```physical_constants```

In [ ]:
sp.constants.physical_constants['Josephson constant']

Per le costanti più utilizzate esistono dei semplici alias, come ad esempio:

In [ ]:
sp.constants.c

Inoltre sono già definite moltissime unità di misura, ad esempio:

In [ ]:
# Il valore che si ottiene è riferito all'unità di misura nel sistema internazionale, in questo caso il metro
sp.constants.yard

Per visualizzare una lista di questi alias si può semplicemente fare:

In [ ]:
dir(sp.constants)

## SciPy & Sparse Data
Per *sparse data* si intende un insieme di dati che ha per di più elementi non utilizzati (i.e. elementi che non trasmettono nessuna informazione).

Come per esempio array del tipo ```[1, 0, 2, 0, 0, 3, 0, 0, 0, 0, 0, 0]```, dove gli ```0``` non esprimono nessuna informazione in particolare.

In analisi dei dati e machine learning spesso ci si può trovare a lavorare con dati sparsi, quindi SciPy fornisce il pacchetto ```sparse```, che implementa alcuni metodi per gestire in maniera più efficiente e agevole dati sparsi.

Il metodo di seguito permette di creare una matrice CSR (Compressed Sparse Row) a partire da un array:

In [ ]:
arr = np.array([0, 0, 0, 0, 0, 1, 1, 0, 2])

csr = sp.sparse.csr_matrix(arr)
csr

Come si può vedere l'oggetto restituito è un wrapper scipy di una matrice sparsa: questo serve proprio per velocizzare delle operazioni quando si usano le funzioni apposite di scipy. Infatti:

In [ ]:
type(csr)

Per stamparla bisogna fare esplicitamente il ```print``` (è stato fato l'*override* della funzione print):

In [ ]:
print(csr)

Questa rappresentazione vuol dire:
* l'oggetto n. 1 è alla riga 0 in posizione 5 e ha valore 1
* l'oggetto n. 2 è alla riga 0 in posizione 6 e ha valore 1
* l'oggetto n. 3 è alla riga 0 in posizione 8 e ha valore 2

Per rappresentare i dati omettendo gli zeri:


In [ ]:
csr.data

Per contare gli elementi diversi da zero:

In [ ]:
csr.count_nonzero()

Per cambiare la rappresentazione da CSR a CSC (Compressed Sparse Column):


In [ ]:
arr = np.array([[0, 0, 0], [0, 0, 1], [1, 0, 2]])

csr = sp.sparse.csr_matrix(arr)
print(csr)

In [ ]:
csc = csr.tocsc()
type(csc)

In [ ]:
print(csc)

## SciPy Graphs

In SciPy esiste anche un sottomodulo di ```sparse``` che permette di lavorare agevolemnte con i grafi. Tutte le operazioni vengono fatte fornendo la matrice di adiacenza del grafo, che deve essere nota a priori.

Si consideri il grafico dato dalla matrice di adiacenza:
$$
\left(\begin{array}{ccc} 
0 & 1 & 2 & 1\\
1 & 0 & 1 & 1\\
2 & 1 & 0 & 0\\
1 & 1 & 0 & 0\\
\end{array}\right)
$$ 

Che corrisponde al grafo:

In [ ]:
adjacency_matrix = np.array([
  [0, 1, 2, 1],
  [1, 0, 1, 1],
  [2, 1, 0, 0],
  [1, 1, 0, 0]
])

# Il codice seguente serve solo a creare una visualizzazione del grafo

import networkx as nx
# Create a graph from the adjacency matrix
G = nx.from_numpy_array(adjacency_matrix)

# Extract edge weights
edge_labels = {(i, j): G[i][j]['weight'] for i, j in G.edges() if 'weight' in G[i][j]}

# Draw the graph
pos = nx.spring_layout(G)  # you can choose different layout algorithms
nx.draw(G, pos, with_labels=True, font_weight='bold', node_size=700, node_color='skyblue', font_color='black', font_size=12)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=12);

Si può esprimere ad esempio la matrice di adiacenza come matrice csr per poi effettuare delle operazioni. Di seguito alcuni esempi.

In [ ]:
csr_graph = sp.sparse.csr_matrix(adjacency_matrix)

# Per ottenere le componenti connesse:
sp.sparse.csgraph.connected_components(csr_graph)

In [ ]:
# Per trovare il cammino minimo da un elemento all'altro con l'algoritmo di Dijkstra
sp.sparse.csgraph.dijkstra(csr_graph)


In [ ]:
# Analogo a prima, ma con l'algoritmo di Floyd Warshall
sp.sparse.csgraph.floyd_warshall(csr_graph)

## Scipy Spatial Data
Per tutti i problemi di tipo spaziale (es. determinare se un punto è interno rispetto a un particolare bordo) SciPy fornisce il modulo ```spatial```.

Di seguito alcuni esempi.

Per effettuare la triangolazione di un poligono avente per vertici un set di punti dati:

In [ ]:
points = np.array([
  [2, 4],
  [3, 4],
  [3, 0],
  [2, 2],
  [4, 1]
])

simplices = sp.spatial.Delaunay(points).simplices

plt.triplot(points[:, 0], points[:, 1], simplices)
plt.scatter(points[:, 0], points[:, 1], color='r')

Per trovare il più piccolo poligono che copra una serie di punti:

In [ ]:
points = np.array([
  [2, 4],
  [3, 4],
  [3, 0],
  [2, 2],
  [4, 1],
  [1, 2],
  [5, 0],
  [3, 1],
  [1, 2],
  [0, 2]
])

hull = sp.spatial.ConvexHull(points)
hull_points = hull.simplices

plt.scatter(points[:,0], points[:,1])
for simplex in hull_points:
  plt.plot(points[simplex,0], points[simplex,1], 'k-')

Per problemi di nearest neighbour:

In [ ]:
points = [(1, -1), (1, 0), (2, 3), (-2, 3), (2, -3)]

kdtree = sp.spatial.KDTree(points)

res = kdtree.query((1, 1))

# Il risultato ha la forma (distanza, indice del punto più vicino)
res

Infine, in questo modulo esistono parecchie metriche per calcolare la distanza. Di seguito qualche esempio:

In [ ]:
p1 = (1, 0)
p2 = (10, 2)

# Distanza euclidea
sp.spatial.distance.euclidean(p1, p2)

In [ ]:
# Distanza Cityblock (Manhattan Distance)
sp.spatial.distance.cityblock(p1, p2)

In [ ]:
# Distanza coseno
sp.spatial.distance.cosine(p1, p2)

In [ ]:
# Distanza di Hamming (frazione di bit che sono diversi)

p1 = (True, False, True)
p2 = (False, True, True)

sp.spatial.distance.hamming(p1, p2)

## SciPy Interpolation

In SciPy sono anche presenti numerosi metodi per fare interpolazione di punti nel modulo ```interpolate```.

Ad esempio, per fare interpolazioni in una dimensione, si usa ```interp1d```, che prende una serie di punti con valori x e y e ritorna una *callable function* che può essere chiamata con i nuovi x e ritorna i nuovi y

In [ ]:
xs = np.arange(10)
ys = 2*xs + 1

interp_func = sp.interpolate.interp1d(xs, ys)

new_x = np.arange(2.1, 3, 0.1)
new_y = interp_func(new_x)

plt.plot(xs, ys, 'o', label="Originali")
plt.plot(new_x, new_y, 'o', label="Interpolati")

plt.legend();

Esistono vari tipi di interpolazione: un altro esempio che si riporta è quello della *spline interpolation*, dove a differenza dell'interpolazione classica che fitta un polinomio di altro grado su tutti i punti, qui vengono fittati polinomi di basso grado su dei subset di punti. Di seguito un esempio:

In [ ]:
xs = np.arange(10)
ys = xs**2 + np.sin(xs) + 1

interp_func = sp.interpolate.UnivariateSpline(xs, ys)

new_x = np.arange(2.1, 3, 0.1)
new_y = interp_func(new_x)

plt.plot(xs, ys, 'o', label="Originali")
plt.plot(new_x, new_y, 'o', label="Interpolati")

plt.legend();

## SciPy Optimizers

Il modulo ```scipy.optimize``` contiene varie funzioni utili a trovare le soluzioni di un'equazione o il minimo di una funzione. Di seguito qualche esempio di utilizzo.

### Soluzioni di equazioni

SciPy può trovare agevolmente soluzioni di equazioni anche non lineari con la funzione ```root``` del pacchetto ```optimize```. Si utilizza una funzione python per rappresentare l'equazione da risolvere e si passa una guess iniziale per la radice dell'equazione. Di seguito un semplice esempio:

In [ ]:
def eqn(x):
  return x + np.cos(x)

myroot = sp.optimize.root(eqn, 0)

myroot

Come si vede l'oggetto ritornato ha vari attributi. La soluzione è contenuta nell'attributo ```x```:

In [ ]:
myroot.x

Possono essere usati diversi solver per trovare le radici di un'equazione. Trattarli singolarmente va oltre lo scopo di questo corso, ma potete approfondire guardando la [documentazione](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html).

### Minimizzazione di funzioni

Per minimizzare una funzione si può utilizzare la funzione ```minimize()```. Similmente a prima, gli argomenti obbligatori sono una funzione python che descriva la funzione da minimizzare e una guess.

Anche per questo metodo sono presenti diversi solver, per maggiori dettagli visitare la [documentazione](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html).

In [ ]:
def eqn(x):
  return x**2 + x + 2

mymin = sp.optimize.minimize(eqn, 0)

mymin

##### Approfondimento: Callback function

Entrambi i metodi visti fino ad ora supportano una funzione di callback che venga eseguita dopo ogni iterazione del processo di ottimizzazione. La funzione di callback deve avere la signature ```callback(OptimizeResult: intermediate_result)```, dove ```intermediate_result``` rappresenta il risultato intermedio del processo di ottimizzazione. Di seguito un esempio che stampa il risultato intermedio dopo ogni iterazione.

In [ ]:
mymin = sp.optimize.minimize(eqn, 0, callback=lambda intermediate_result : print(intermediate_result))

In [ ]:
# Si verifica facilmente che il risultato effettivamente si è ottenuto in due iterazioni controllando l'attributo nit
mymin

### Fitting di curve
L'applicazione per cui forse SciPy è più utile in fisica (e non solo) è proprio per il fitting di vari tipi di dati.

Si supponga di avere il seguente set di dati:

In [ ]:
# colors for plots
colors = ['#c3121e', '#0348a1', '#ffb01c', '#027608', '#0193b0', '#9c5300', '#949c01', '#7104b5']

In [ ]:
xdata = np.array([ -10.0, -9.0, -8.0, -7.0, -6.0, -5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0])
ydata = np.array([1.2, 4.2, 6.7, 8.3, 10.6, 11.7, 13.5, 14.5, 15.7, 16.1, 16.6, 16.0, 15.4, 14.4, 14.2, 12.7, 10.3, 8.6, 6.1, 3.9, 2.1])
plt.plot(xdata, ydata, 'o', color=colors[0], label="Data Points")

plt.xlim(-11, 11)
plt.ylim(0, 18)

plt.legend()

plt.grid(True);

Come si può intuire questi dati possono essere approssimati da vari tipi di funzioni. Come primo tentativo proviamo una parabola:

In [ ]:
# funzione per il fit
def parab(x, a, b, c):
    return a * x**2 + b * x + c

popt, pcov = sp.optimize.curve_fit(parab, xdata, ydata)

a = popt[0]
b = popt[1]
c = popt[2]

print(a, b, c)

La variabile ```pcov``` contiene la matrice di covarianza dei parametri fittati:

In [ ]:
pcov

Possiamo ottenere lo standard error e la deviazione standard dalla matrice e plottare i risultati:

In [ ]:
std_error = np.sqrt(np.diag(pcov))
ndata = len(xdata)
sigma = np.sqrt(ndata * np.diag(pcov))

print("a =", a, "+/-", sigma[0])
print("b =", b, "+/-", sigma[1])
print("c =", c, "+/-", sigma[2])

Per valutare la bontà del fit, possiamo molto semplicemente definire una funzione che valuti l'R<sup>2</sup>.

In [ ]:
def R_squared(x, y, func, *params):
    res = y - func(x, *params)
    ss_res = np.sum(res**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    r_squared = 1 - (ss_res/ss_tot)
    return r_squared

R2 = R_squared(xdata, ydata, parab, *popt)
R2

Infine plottiamo i risultati:

In [ ]:
plt.plot(xdata, ydata, 'o', color=colors[0], label="Data Points")

x_fit = np.linspace(xdata[0], xdata[-1], 2000)
plt.plot(x_fit, parab(x_fit, *popt), '-', color=colors[1], label=f"Fit $R^2$ = {R2:.5f}")

plt.xlim(-11, 11)
plt.ylim(0, 18)

plt.legend()

plt.grid(True);

Allo stesso modo si può seguire la stessa procedura per fittare i dati con una gaussiana:

In [ ]:
def gauss(x, A, B):
    y = A*np.exp(-1 * B * x**2)
    return y

popt, pcov = sp.optimize.curve_fit(gauss, xdata, ydata)
sigma = np.sqrt(ndata * np.diag(pcov))

R2 = R_squared(xdata, ydata, gauss, *popt)

print(f"A = {popt[0]} +/- {sigma[0]}")
print(f"B = {popt[1]} +/- {sigma[1]}")
print(f"R^2 = {R2}")

In [ ]:
# Plottiamo il risultato
plt.plot(xdata, ydata, 'o', color=colors[0], label="Data Points")

x_fit = np.linspace(xdata[0], xdata[-1], 2000)
plt.plot(x_fit, gauss(x_fit, *popt), '-', color=colors[1], label=f"Fit gauss $R^2$ = {R2:.5f}")

plt.xlim(-11, 11)
plt.ylim(0, 18)

plt.legend()

plt.grid(True);

Si può provare allo stesso modo con la funzione coseno:

In [ ]:
def cos_func(x, D, E):
    y = D*np.cos(E*x)
    return y

popt, pcov = sp.optimize.curve_fit(cos_func, xdata, ydata)
sigma = np.sqrt(ndata * np.diag(pcov))

R2 = R_squared(xdata, ydata, cos_func, *popt)

print(f"D = {popt[0]} +/- {sigma[0]}")
print(f"E = {popt[1]} +/- {sigma[1]}")
print(f"R^2 = {R2}")

Il risultato è molto brutto. Proviamo a capire che cosa sta succedendo:

In [ ]:
plt.plot(xdata, ydata, 'o', color=colors[0], label="Data Points")

x_fit = np.linspace(xdata[0], xdata[-1], 2000)
plt.plot(x_fit, cos_func(x_fit, *popt), '-', color=colors[1], label=f"Fit cos $R^2$ = {R2:.5f}")

plt.xlim(-11, 11)
plt.ylim(0, 18)

plt.legend()

plt.grid(True);

La procedura di ottimizzazione non riesce a convergere. Proviamo a fornire delle guess iniziali al fit e vediamo se la situazione migliora:

In [ ]:
guess = [16, 0.1]
popt, pcov = sp.optimize.curve_fit(cos_func, xdata, ydata, p0=guess)
sigma = np.sqrt(ndata * np.diag(pcov))

R2 = R_squared(xdata, ydata, cos_func, *popt)

print(f"D = {popt[0]} +/- {sigma[0]}")
print(f"E = {popt[1]} +/- {sigma[1]}")
print(f"R^2 = {R2}")

Ora il risultato è decisamente migliore. Infatti:

In [ ]:
plt.plot(xdata, ydata, 'o', color=colors[0], label="Data Points")

x_fit = np.linspace(xdata[0], xdata[-1], 2000)
plt.plot(x_fit, cos_func(x_fit, *popt), '-', color=colors[1], label=f"Fit cos $R^2$ = {R2:.5f}")

plt.xlim(-11, 11)
plt.ylim(0, 18)

plt.legend()

plt.grid(True);

**OSS**: potete anche fornire dei bounds al fit col parametro ```bounds```, ove sia necessario.

## Esercizi

### Esercizio 1
Prova a fare un'interpolazione dei dati sopra e osserva cosa cambia rispetto a una funzione fittata.

### Esercizio 2
Genera 1000 dati da una distribuzione gaussiana con media e deviazione standard anch'esse randomiche e prova a effettuare un fit. Supponi inoltre che l'errore sulla y dei dati sia dato da:
$$err_i = \sqrt{y_i}$$

Valuta la bontà del fit con il parametro R<sup>2</sup> e plotta infine i dati.